In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import os
#import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
os.chdir(r"/content/drive/MyDrive")

In [ ]:
# define image size
IMAGE_SIZE = [60, 60]

In [ ]:
#Give dataset path
train_path = r'/content/drive/MyDrive/room_datasets/train'
test_path = r'/content/drive/MyDrive/room_datasets/test'

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
# useful for getting number of classes
folders = os.listdir(r"/content/drive/MyDrive/room_datasets/train")
print(len(folders))

3


In [ ]:
# useful for getting number of classes
folders = os.listdir(r"/content/drive/MyDrive/room_datasets/test")
print(len(folders))

3


In [ ]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 60, 60, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 60, 60, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 60, 60, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 30, 30, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 30, 30, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 30, 30, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 15, 15, 128)       0     

In [ ]:
from tensorflow.keras import optimizers


sgd = optimizers.SGD(learning_rate=0.0001, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer="sgd",
              metrics=["accuracy"])

In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
# Data Augmentation
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
# Make sure you provide the same target size as initialied for the image size
train_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (60, 60),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 133 images belonging to 3 classes.


In [ ]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (60, 60),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 36 images belonging to 3 classes.


In [ ]:
import time

def get_log_path(log_dir="logs/fit"):
  uniqueName = time.strftime("log_%Y_%m_%d_%H_%M_%S")
  log_path = os.path.join(log_dir, uniqueName)
  print(f"savings logs at: {log_path}")

  return log_path

log_dir = get_log_path()

savings logs at: logs/fit/log_2022_07_07_17_49_28


In [ ]:
import tensorflow as tf
file_writer = tf.summary.create_file_writer(logdir=log_dir)


In [ ]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True)
CKPT_path = "model_ckpt.h5"

In [ ]:

checkpointing_cb = tf.keras.callbacks.ModelCheckpoint(CKPT_path, save_best_only=True)
CALLBACKS_LIST = [tensorboard_cb, early_stopping_cb, checkpointing_cb]

In [ ]:
history = model.fit(train_set, epochs=200, validation_data=test_set,callbacks=CALLBACKS_LIST)

Epoch 1/200
2/5 [===========>..................] - ETA: 1s - loss: 7.3196 - accuracy: 0.5469

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


5/5 [==============================] - 4s 770ms/step - loss: 17.1126 - accuracy: 0.4511 - val_loss: 47.4384 - val_accuracy: 0.4722
Epoch 2/200
5/5 [==============================] - 4s 672ms/step - loss: 19.9777 - accuracy: 0.5038 - val_loss: 22.2898 - val_accuracy: 0.6111
Epoch 3/200
5/5 [==============================] - 3s 719ms/step - loss: 8.6638 - accuracy: 0.6917 - val_loss: 25.4121 - val_accuracy: 0.5556
Epoch 4/200
5/5 [==============================] - 3s 696ms/step - loss: 10.8348 - accuracy: 0.6692 - val_loss: 28.8965 - val_accuracy: 0.5000
Epoch 5/200
5/5 [==============================] - 4s 736ms/step - loss: 15.6637 - accuracy: 0.5940 - val_loss: 12.7713 - val_accuracy: 0.7222
Epoch 6/200
5/5 [==============================] - 4s 619ms/step - loss: 25.2763 - accuracy: 0.6165 - val_loss: 12.1793 - val_accuracy: 0.7222
Epoch 7/200
5/5 [==============================] - 3s 712ms/step - loss: 8.0420 - accuracy: 0.7293 - val_loss: 14.3858 - val_accuracy: 0.6667
Epoch 8/200
5

In [ ]:
import time
import os

def saveModel_path(model_dir="SAVED_MODELS"):
    os.makedirs(model_dir, exist_ok=True)
    fileName = time.strftime("Model_%Y_%m_%d_%H_%M_%S_.h5")    
    model_path = os.path.join(model_dir, fileName)
    print(f"your model will be saved at the following location\n{model_path}")
    return model_path

In [ ]:
model.save(saveModel_path())

your model will be saved at the following location
SAVED_MODELS/Model_2022_07_07_18_05_10_.h5
